### Filtrando e organizando datagrames de paradas de onibus

A primeira parte do projeto consiste em criar funções para leituras de arquivos shapefiles

In [1]:
# Abre o dataframe corretamente
FILE_PARADAS = "../../../databases/paradas/paradas.shp"
FILE_LINHAS = "../../../databases/linhas/Linhas.shp"

import geopandas as gpd

def abreSHP(caminhoSHP):

    dados_shapefile = gpd.read_file(caminhoSHP)

    return dados_shapefile

df_linhas = abreSHP(FILE_LINHAS)
df_paradas = abreSHP(FILE_PARADAS)
df_linhas = df_linhas.to_crs(epsg=4326)
df_paradas = df_paradas.to_crs(epsg=4326)
num_linha = '0.205'
df_linhas[df_linhas['linha']== num_linha].geometry

In [10]:

linha_813 = df_linhas[(df_linhas['linha'] == num_linha) & (df_linhas['sentido'] == 'VOLTA')]

In [11]:
import geopandas as gpd
from shapely.geometry import Point
import json


FILE_PARADAS = "../../../databases/paradas/paradas.shp"
FILE_LINHAS = "../../../databases/linhas/Linhas.shp"


def abreSHP(caminhoSHP):

    dados_shapefile = gpd.read_file(caminhoSHP)

    return dados_shapefile

df_linhas = abreSHP(FILE_LINHAS)
df_paradas = abreSHP(FILE_PARADAS)
df_linhas = df_linhas.to_crs(epsg=4326)
df_paradas = df_paradas.to_crs(epsg=4326)
num_linha = '0.205'
df_linhas[df_linhas['linha']== num_linha].geometry


# Filtrar a linha de ônibus '0.813' do DataFrame df_linhas.
linha_813 = df_linhas[(df_linhas['linha'] == num_linha ) & (df_linhas['sentido'] == 'VOLTA')]

# Crie um dicionário para armazenar as paradas que fazem parte da rota na ordem de comparação.
paradas_na_rota = {}

# Define a tolerância de proximidade em graus decimais.
tolerancia = 0.00019  # Cerca de 10 metros em graus decimais (para EPSG:4326).

# Itere pelos pontos da linha de ônibus.
for idx, ponto_linha in linha_813.iterrows():
    ponto_linha_geometry = ponto_linha['geometry']
    # Itere pelas paradas.
    for idx_parada, parada in df_paradas.iterrows():
        parada_geometry = parada['geometry']
        # Calcule a distância entre o ponto da linha de ônibus e a parada em graus decimais.
        distancia = ponto_linha_geometry.distance(parada_geometry)
        
        # Se a distância for menor que a tolerância, considere a parada parte da rota.
        if distancia < tolerancia:
            # Armazene a parada no dicionário na ordem em que foram comparadas.
            paradas_na_rota[idx_parada] = {
                'latitude': parada_geometry.y,
                'longitude': parada_geometry.x
            }

# O dicionário 'paradas_na_rota' agora contém as paradas que fazem parte da rota da linha de ônibus '0.813' na ordem de comparação.
# Define o nome do arquivo JSON de saída.
output_json_filename = 'paradas_na_rota_em_ordem.json'

# Salva o dicionário em um arquivo JSON.
with open(output_json_filename, 'w') as json_file:
    json.dump(paradas_na_rota, json_file)

print("Paradas na rota salvas em 'paradas_na_rota_em_ordem.json'.")

Paradas na rota salvas em 'paradas_na_rota_em_ordem.json'.


In [7]:
### Modificado para todas as linhas de onibus

import geopandas as gpd
from shapely.geometry import Point
import json

FILE_PARADAS = "../../../databases/paradas/paradas.shp"
FILE_LINHAS = "../../../databases/linhas/Linhas.shp"


def abreSHP(caminhoSHP):
    dados_shapefile = gpd.read_file(caminhoSHP)
    return dados_shapefile

df_linhas = abreSHP(FILE_LINHAS)
df_paradas = abreSHP(FILE_PARADAS)
df_linhas = df_linhas.to_crs(epsg=4326)
df_paradas = df_paradas.to_crs(epsg=4326)

# Crie um dicionário para armazenar as paradas de todas as linhas de ônibus.
todas_as_paradas = {}

# Define a tolerância de proximidade em graus decimais.
tolerancia = 0.00019  # Cerca de 10 metros em graus decimais (para EPSG:4326).

# Itere pelas linhas de ônibus únicas.
for num_linha in df_linhas['linha'].unique():
    # Filtrar a linha de ônibus atual do DataFrame df_linhas.
    linha_atual = df_linhas[(df_linhas['linha'] == num_linha) & (df_linhas['sentido'] == 'CIRCULAR')]

    # Crie um dicionário para armazenar as paradas que fazem parte da rota da linha atual na ordem de comparação.
    paradas_na_rota = {}

    # Itere pelos pontos da linha de ônibus atual.
    for idx, ponto_linha in linha_atual.iterrows():
        ponto_linha_geometry = ponto_linha['geometry']
        # Itere pelas paradas.
        for idx_parada, parada in df_paradas.iterrows():
            parada_geometry = parada['geometry']
            # Calcule a distância entre o ponto da linha de ônibus e a parada em graus decimais.
            distancia = ponto_linha_geometry.distance(parada_geometry)

            # Se a distância for menor que a tolerância, considere a parada parte da rota.
            if distancia < tolerancia:
                # Armazene a parada no dicionário na ordem em que foram comparadas.
                paradas_na_rota[idx_parada] = {
                    'latitude': parada_geometry.y,
                    'longitude': parada_geometry.x
                }

    # Adicione o dicionário de paradas da linha atual ao dicionário geral de todas as paradas.
    todas_as_paradas[num_linha] = paradas_na_rota

# O dicionário 'todas_as_paradas' agora contém as paradas que fazem parte da rota de todas as linhas de ônibus na ordem de comparação.
# Define o nome do arquivo JSON de saída.
output_json_filename = 'paradas_linhas_circular.json'

# Salva o dicionário em um arquivo JSON.
with open(output_json_filename, 'w') as json_file:
    json.dump(todas_as_paradas, json_file)


In [10]:
import json

# Carregue o JSON das rotas de linhas.
with open('paradas_linhas_circular.json', 'r') as json_file:
    todas_as_paradas = json.load(json_file)

# Defina o sentido como "ida" para todas as linhas no JSON.
for paradas_na_rota in todas_as_paradas.values():
    for parada_info in paradas_na_rota.values():
        parada_info['sentido'] = 'circular'

# Agora, o JSON 'todas_as_paradas_na_rota_em_ordem.json' foi atualizado com o sentido "ida".
# Você pode salvar o JSON atualizado de volta ao arquivo, se desejar.
with open('paradas_linhas_circular.json', 'w') as json_file:
    json.dump(todas_as_paradas, json_file)

print("Sentido 'ida' definido para todas as linhas e JSON atualizado.")


Sentido 'ida' definido para todas as linhas e JSON atualizado.


In [5]:
import folium
import json

# Substitua 'todas_as_paradas_na_rota_em_ordem.json' pelo caminho do seu arquivo JSON das linhas.
caminho_json_linhas = 'todas_as_paradas_na_rota_em_ordem.json'

# Substitua '0.205' pela linha específica que você deseja plotar.
linha_especifica = '0.813'

# Carregue o JSON com as informações das linhas.
with open(caminho_json_linhas, 'r') as json_file:
    todas_as_paradas = json.load(json_file)

# Verifique se a linha específica está presente no JSON.
if linha_especifica in todas_as_paradas:
    paradas_na_rota = todas_as_paradas[linha_especifica]

    # Crie um mapa no centro das coordenadas iniciais (por exemplo, a primeira parada da linha específica).
    latitude_inicial = next(iter(paradas_na_rota.values()))['latitude']
    longitude_inicial = next(iter(paradas_na_rota.values()))['longitude']
    m = folium.Map(location=[latitude_inicial, longitude_inicial], zoom_start=15)

    # Crie uma lista de coordenadas para a linha específica.
    coordenadas_linha = [(info['latitude'], info['longitude']) for info in paradas_na_rota.values()]

    # Adicione uma linha polilínea para representar a rota da linha específica.
    #folium.PolyLine(locations=coordenadas_linha, color='blue', weight=3, opacity=0.7).add_to(m)

    # Adicione marcadores para cada parada na rota.
    for parada_id, info in paradas_na_rota.items():
        folium.Marker([info['latitude'], info['longitude']], tooltip=f'Parada {parada_id}').add_to(m)

    # Exiba o mapa.
m


In [15]:
import folium
import json

# Carregue o JSON com as paradas na rota.
with open('paradas_na_rota_final.json', 'r') as json_file:
    paradas_na_rota = json.load(json_file)

# Crie um mapa no centro das coordenadas iniciais (por exemplo, a primeira parada).
latitude_inicial = next(iter(paradas_na_rota.values()))['latitude']
longitude_inicial = next(iter(paradas_na_rota.values()))['longitude']
m = folium.Map(location=[latitude_inicial, longitude_inicial], zoom_start=15)

for parada_id, info in paradas_na_rota.items():
    folium.Marker([info['latitude'], info['longitude']], tooltip=f'Parada {parada_id}').add_to(m)

# Exiba o mapa.
m


In [12]:
import folium
import json

# Carregue o JSON com as paradas na rota.
with open('paradas_na_rota_em_ordem.json', 'r') as json_file:
    paradas_na_rota = json.load(json_file)

# Crie um mapa no centro das coordenadas iniciais (por exemplo, a primeira parada).
latitude_inicial = next(iter(paradas_na_rota.values()))['latitude']
longitude_inicial = next(iter(paradas_na_rota.values()))['longitude']
m = folium.Map(location=[latitude_inicial, longitude_inicial], zoom_start=15)

# Adicione marcadores para as paradas na rota no mapa.
for parada_id, info in paradas_na_rota.items():
    folium.Marker([info['latitude'], info['longitude']], tooltip=f'Parada {parada_id}').add_to(m)

# Exiba o mapa.
m


In [24]:
import json
from shapely.geometry import Point

# Carregue o arquivo JSON de paradas na rota.
with open('paradas_na_rota_em_ordem.json', 'r') as json_file:
    paradas_na_rota = json.load(json_file)

# Converta o dicionário de paradas em uma lista para que possamos iterar por todas.
paradas_na_rota_lista = list(paradas_na_rota.items())

# Defina a tolerância de proximidade em graus decimais.
tolerancia = 0.0009  # Cerca de 10 metros em graus decimais (para EPSG:4326).

# Inicialize uma lista para armazenar as paradas finais.
paradas_finais = [paradas_na_rota_lista[0]]  # Comece com a primeira parada.

# Itere pelas paradas a partir da segunda parada.
for i in range(1, len(paradas_na_rota_lista)):
    parada_atual_idx, parada_atual = paradas_na_rota_lista[i]
    parada_atual_geometry = Point(parada_atual['longitude'], parada_atual['latitude'])
    
    # Verifique a distância entre a parada atual e todas as paradas finais.
    distancia_menor_que_10 = any(parada_atual_geometry.distance(Point(p[1]['longitude'], p[1]['latitude'])) < tolerancia for p in paradas_finais)
    
    if not distancia_menor_que_10:
        paradas_finais.append((parada_atual_idx, parada_atual))

# Converta a lista de paradas finais de volta para um dicionário.
paradas_finais_dict = dict(paradas_finais)

# Atualize o arquivo JSON com as paradas finais.
with open('paradas_na_rota_final.json', 'w') as json_file:
    json.dump(paradas_finais_dict, json_file)

print("Paradas na rota final salvas em 'paradas_na_rota_final.json'.")


Paradas na rota final salvas em 'paradas_na_rota_final.json'.


In [14]:
import json
from shapely.geometry import Point
from sklearn.cluster import DBSCAN
import numpy as np

# Carregue o arquivo JSON de paradas na rota.
with open('paradas_na_rota_em_ordem.json', 'r') as json_file:
    paradas_na_rota = json.load(json_file)

# Extraia as coordenadas (latitude e longitude) das paradas.
coordenadas = [(p['latitude'], p['longitude']) for p in paradas_na_rota.values()]

# Converta as coordenadas em um array NumPy.
X = np.array(coordenadas)

# Defina a distância máxima para considerar paradas como parte do mesmo cluster.
distancia_maxima = 0.0015  # Ajuste conforme necessário.

# Crie um modelo DBSCAN com a distância máxima especificada.
dbscan = DBSCAN(eps=distancia_maxima, min_samples=1)

# Realize o agrupamento das paradas.
labels = dbscan.fit_predict(X)

# Inicialize um dicionário para armazenar as paradas finais.
paradas_finais = {}

# Itere pelas paradas e mantenha apenas uma de cada cluster.
for cluster_id in set(labels):
    # Encontre as paradas pertencentes a este cluster.
    cluster_indices = np.where(labels == cluster_id)[0]
    
    # Escolha uma parada do cluster (por exemplo, a primeira).
    parada_escolhida_idx = cluster_indices[0]
    parada_escolhida = paradas_na_rota[list(paradas_na_rota.keys())[parada_escolhida_idx]]
    
    # Converta o índice para uma string e adicione a parada escolhida ao dicionário de paradas finais.
    paradas_finais[str(parada_escolhida_idx)] = parada_escolhida

# Atualize o arquivo JSON com as paradas finais.
with open('paradas_na_rota_final.json', 'w') as json_file:
    json.dump(paradas_finais, json_file)

print("Paradas na rota final salvas em 'paradas_na_rota_final.json'.")


Paradas na rota final salvas em 'paradas_na_rota_final.json'.


In [152]:
# Converte para o formato correto

def converter_para_EPSG_4326(point):
    from shapely.geometry import Point
    import pyproj

    transformer = pyproj.Transformer.from_crs("epsg:31983", "epsg:4326", always_xy=True)
    
    lon, lat = transformer.transform(point.x, point.y)
    
    ponto_EPSG_4326 = Point(lon, lat)
    
    return ponto_EPSG_4326


In [153]:
from shapely.geometry import Point
from shapely.ops import transform
from functools import partial
import pyproj

def calcular_distancia_km(point1, point2):
    project_to_EPSG_4326 = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:31983'),  # Projeto de origem (EPSG 31983)
        pyproj.Proj(init='epsg:4326'),  # Projeto de destino (EPSG 4326)
    )

    # Transforma os pontos para o EPSG 4326
    point1_wgs84 = transform(project_to_EPSG_4326, point1)
    point2_wgs84 = transform(project_to_EPSG_4326, point2)

    # Calcula a distância entre os pontos em graus
    distancia_graus = point1_wgs84.distance(point2_wgs84)

    # Converte a distância para metros (considerando que 1 grau de latitude ~= 111.32 km)
    distancia_metros = distancia_graus * 111320.0

    return distancia_metros



In [154]:
# Plota gráficos para a gente
import folium

def criar_mapa_com_dois_pontos(coord1, coord2):
    # Cria um objeto de mapa
    m = folium.Map(location=coord1, zoom_start=15)

    # Adiciona um círculo azul para o primeiro ponto
    folium.CircleMarker(
        location=coord1,
        radius=10,  # Tamanho do círculo em pixels
        color='blue',  # Cor do círculo
        fill=True,
        fill_color='blue',  # Cor de preenchimento do círculo
        fill_opacity=0.6,
    ).add_to(m)

    # Adiciona um círculo azul para o segundo ponto
    folium.CircleMarker(
        location=coord2,
        radius=10,  # Tamanho do círculo em pixels
        color='blue',  # Cor do círculo
        fill=True,
        fill_color='blue',  # Cor de preenchimento do círculo
        fill_opacity=0.6,
    ).add_to(m)

    return m

# Exemplo de uso da função
coordenadas1 = [-15.91131954437385, -47.76791017769213]
coordenadas2 = [-15.911596647471995, -47.76792780490262]

mapa = criar_mapa_com_dois_pontos(coordenadas1, coordenadas2)

# Salva o mapa como um arquivo HTML
mapa.save("mapa_com_dois_pontos.html")
m

In [155]:
# Cacular desvio padrão com relação as médias


In [156]:
# Armazenar as distancias.


In [157]:
# Remover paradas e deixar dataframe das paradas em ordem